In [ ]:
from __future__ import division
import pyomo.environ as pyomo
import pyomo.opt as opt

# Welcome to the TESA programming exercise on basic dispatch LP models

## 1. Prepare the input data for the LP model

In [ ]:
# introduce sets of indices
G = ['CCGT','GT','Oil','Hydro','Hard coal','Lignite','Nuclear','Wind'] # generator technologies

In [ ]:
# maximal CO2 emissions [Mt]
MAX_CO2 = 360

In [ ]:
# Demand for electric energy [TWh]
DEMAND = 620 

In [ ]:
# Technical specifications of the generator technologies

# convention:   tech_data[(g,'capacity')] = maximal electricity output of technology g [TWh/a]
#               tech_data[(g,'eta_el')] = eta_electric of technology g [1]
#               tech_data[(g,'fuel price')]= fuel price of technology g [€/MWh_thermal]
#               tech_data[(g,'other variable costs')] = other variable costs of technology g [€/MWh_el]
#               tech_data[(g,'emission factor')] = emission factor of technology g [g CO2/kWh_thermal]
tech_data_g = {('CCGT','capacity'):      149.7,  ('CCGT','eta_el'):      0.54,   ('CCGT','fuel price'):      25.8,   ('CCGT','other variable costs'):      1.5,    ('CCGT','emission factor'):       204.8, 
               ('GT','capacity'):        7.0,    ('GT','eta_el'):        0.28,   ('GT','fuel price'):        25.8,   ('GT','other variable costs'):        1.5,    ('GT','emission factor'):         204.8, 
               ('Oil','capacity'):       13.9,   ('Oil','eta_el'):       0.28,   ('Oil','fuel price'):       50.1,   ('Oil','other variable costs'):       1.7,    ('Oil','emission factor'):        266.4, 
               ('Hydro','capacity'):     27.1,   ('Hydro','eta_el'):     1.0,    ('Hydro','fuel price'):     0.0,    ('Hydro','other variable costs'):     1.5,    ('Hydro','emission factor'):      0.0,
               ('Hard coal','capacity'): 193.2,  ('Hard coal','eta_el'): 0.41,   ('Hard coal','fuel price'): 7.7,    ('Hard coal','other variable costs'): 2.6,    ('Hard coal','emission factor'):  342.0, 
               ('Lignite','capacity'):   168.7,  ('Lignite','eta_el'):   0.38,   ('Lignite','fuel price'):   3.8,    ('Lignite','other variable costs'):   3.0,    ('Lignite','emission factor'):    399.6, 
               ('Nuclear','capacity'):   157.3,  ('Nuclear','eta_el'):   0.33,   ('Nuclear','fuel price'):   1.8,    ('Nuclear','other variable costs'):   0.7,    ('Nuclear','emission factor'):    0.0,
               ('Wind','capacity'):      27.6,   ('Wind','eta_el'):      1.0,    ('Wind','fuel price'):      0.0,    ('Wind','other variable costs'):      1.5,    ('Wind','emission factor'):       0.0}

## 2. Build the LP model

In [ ]:
model = pyomo.ConcreteModel()

### 2.1 Define Sets

In [ ]:
# define sets of the LP model  
model.G = pyomo.Set(initialize=G)

### 2.2 Define Variables

In [ ]:
# create decision variables
model.x_g = pyomo.Var(model.G, domain=pyomo.NonNegativeReals) # electricity output of technology g [TWh]

### 2.3 Define Constraints

In [ ]:
# cover demand
def define_demand_restriction(model):
    return sum(model.x_g[g] for g in model.G) == DEMAND
model.demand_restriction = pyomo.Constraint(rule=define_demand_restriction)

In [ ]:
# capacity restriciton
def define_capacity_restriciton(model, g):
    return model.x_g[g] <= tech_data_g[(g,'capacity')]
model.capacity_restriciton = pyomo.Constraint(model.G, rule=define_capacity_restriciton)

In [ ]:
# environmental restriction
def define_environmental_restricion(model):
    return sum((tech_data_g[(g,'emission factor')]/(1000*tech_data_g[(g,'eta_el')]))*model.x_g[g] for g in model.G) <= MAX_CO2
model.environmental_restricion = pyomo.Constraint(rule=define_environmental_restricion)

### 2.4 Define Objective Function

In [ ]:
def define_objective_function(model):
    return sum((tech_data_g[(g,'fuel price')]/tech_data_g[(g,'eta_el')]+tech_data_g[(g,'other variable costs')])*model.x_g[g] for g in model.G)
model.Obj = pyomo.Objective(rule=define_objective_function, sense=pyomo.minimize)

### 2.5 Write LP to File

In [ ]:
model.write('output/dispatch/04_esa_uebung_LP1_dispatch_loesung.lp', io_options={'symbolic_solver_labels':True})

## 3. Solve the LP model

In [ ]:
optimizer = opt.SolverFactory('glpk')
solved_model = optimizer.solve(model, tee=True)

## 4. Get the results and statistics of the solved LP model

### 4.1 print optimal objective value

In [ ]:
display("Optimal value: %.2f Mio. €" % (round(model.Obj.expr(),2)))

### 4.2 print optimal dispatch path

In [ ]:
EPS = 1.e-6    # definition of threshold to avoid numerical issues

for g in G:
    if pyomo.value(model.x_g[g]) > EPS:
        print("Dispatch %i TWh of technology %s" % (pyomo.value(model.x_g[g]),g))